# Box Functions

## Random Boxes

These boxes are created to ensure that the top < bottom and left < right.

In [ ]:
def make_random_boxes(num_boxes=1):
    pair_1 = tf.random.uniform((num_boxes, 2))
    pair_2 = tf.random.uniform((num_boxes, 2))

    yx_min = tf.where(pair_1 < pair_2, pair_1, pair_2)
    yx_max = tf.where(pair_1 > pair_2, pair_1, pair_2)

    boxes = tf.concat([yx_min, yx_max], axis=-1)
    return tf.RaggedTensor.from_tensor(boxes)

def make_random_grid_boxes(num_boxes, grid_size):
    """
        Generates a random boxes in a grid.

        Arguments:
            num_boxes: An integer value to indicate the number of boxes to generate.
            grid_size: The size of target grid to fit the boxes into.
        
        Returns:
            A grid containing heights and widths of input boxes fitted based on their top-left coordinates.
            The output shape is (grid_size, grid_size, 2)
    """
    pair_1 = tf.random.uniform((num_boxes, 2))
    pair_2 = tf.random.uniform((num_boxes, 2))

    yx_min = tf.where(pair_1 < pair_2, pair_1, pair_2)
    yx_max = tf.where(pair_1 > pair_2, pair_1, pair_2)

    boxes = tf.concat([yx_min, yx_max], axis=-1)
    box_grid = yxyx_to_hw_grid(boxes, grid_size)

    # box_indices = tf.cast(tf.random.uniform((num_boxes, 2))*grid_size, dtype=tf.int32)
    # box_grid = tf.scatter_nd(indices=box_indices, updates=boxes, shape=(grid_size, grid_size, 4))

    # tf.print('box_indices: ', box_indices, box_indices.shape)
    # tf.print('boxes: ', boxes, boxes.shape)
    # tf.print('box_grid: ', box_grid, box_grid.shape)

    return box_grid

## Box Translations

Translation between YXYX, YXHW and Grid HW formats.

In [ ]:
def generate_grid_indices(size):
    def cell_indices():
        indices = tf.repeat(tf.range(size), size)
        return tf.reshape(indices, (size, size))
    
    row_ids = cell_indices()
    col_ids = tf.transpose(row_ids)
    result = tf.concat([tf.expand_dims(row_ids, axis=-1), tf.expand_dims(col_ids, axis=-1)], axis=-1)
    return result

def yxyx_to_yxhw(boxes):
    """
        It translates the boxes from YXYX format to YXHW format.
        boxes: A tensor of boxes with shape (N_BOXES, 4) in YXYX format.

        Returns:
            A tensor of boxes with shape (N_BOXES, 4) in YXHW format.
    """
    yx_min, yx_max = tf.split(boxes, 2, axis=-1)
    hw = yx_max - yx_min
    hw_boxes = tf.concat([yx_min, hw], axis=-1)
    return hw_boxes

def yxhw_to_yxyx(boxes):
    """
        It translates the boxes from YXHW format to YXYX format.
        boxes: A tensor of boxes with shape (N_BOXES, 4) in YXHW format.

        Returns:
            A tensor of boxes with shape (N_BOXES, 4) in YXYX format.
    """
    yx_min, yx_hw = tf.split(boxes, 2, axis=-1)
    yx_max = tf.clip_by_value(yx_min + yx_hw, 0, 1)
    yxyx_boxes = tf.concat([yx_min, yx_max], axis=-1)
    return yxyx_boxes

## Represenation Translations

Translations between Dense, Sparse and Ragged boxes.

In [ ]:
def sparse_to_ragged_boxes(sparse_boxes, size):
    yx_min = tf.cast(tf.reshape(sparse_boxes.indices, [-1, 6])[:, :2]/size, dtype=sparse_boxes.dtype)
    yx_max = tf.reshape(sparse_boxes.values, [-1, 2])

    # tf.print('yx_min: ', yx_min, yx_min.shape)
    # tf.print('yx_max: ', yx_max, yx_max.shape)

    boxes = tf.concat([yx_min, yx_max], axis=-1)

    return boxes

def sparsify_boxes(bboxes, size):
    bboxes = bboxes.to_tensor()
    num_boxes = tf.shape(bboxes)[0]

    # Extract box properties.
    [yx_min, yx_max] = tf.split(bboxes, 2, axis=-1)

    # Compute box dimensions which includes their heights and widths
    hw = yx_max - yx_min

    # Compute (y_min, x_min) indices of the boxes to address their top-left corner.
    yx_indices = yx_to_indices(yx_min, size)

    # Compute indices for sparse tensor of shape (IMG_SIZE, IMG_SIZE, 2) as [[Y, X, H], [Y, X, W]...].
    sparse_yx_indices = tf.repeat(yx_indices, 2, axis=0)
    sparse_hw_indices = tf.reshape(tf.repeat(tf.constant([[0, 1]], dtype=tf.int64), num_boxes, axis=0), [-1, 1])
    sparse_indices = tf.concat([sparse_yx_indices, sparse_hw_indices], axis=-1)
    sparse_values = tf.reshape(hw, [-1])

    sparse_boxes = tf.sparse.SparseTensor(indices=sparse_indices, values=sparse_values, dense_shape=(size, size, 2))

    return sparse_boxes

# boxes_yxyx_1 = tf.SparseTensor(indices=[[0, 1, 0], [0, 1, 1]], values=[.25, .5], dense_shape=[IMG_SIZE, IMG_SIZE, 2])
# boxes_yxyx_2 = tf.SparseTensor(indices=[[100, 100, 0], [100, 100, 1]], values=[.75, .95], dense_shape=[IMG_SIZE, IMG_SIZE, 2])

# sparse_to_ragged_boxes(boxes_yxyx_1, IMG_SIZE)

## Box Properties

In [ ]:
def box_area(boxes):
    """
        It computes the area of the boxes.
        boxes: A tensor of boxes with shape (N_BOXES, 4) in YXYX format.

        Returns:
            A tensor of box areas of shape (N_BOXES, 1)
    """
    y_min, x_min, y_max, x_max = tf.split(boxes, 4, axis=-1)
    return (y_max - y_min)*(x_max - x_min)